In [2]:
# libraries required, please install pandas
import pandas as pd
from unidecode import unidecode
import re
import datetime
from datetime import date
import json
import numpy as np
import os
import time
# set column options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [103]:
j_data=pd.read_excel('./031_recon/j_data_final.xlsx')
cit_network=pd.read_excel('./031_recon/network_cit_1940_2020.xlsx')
all_refs=pd.read_excel('./031_recon/refs_1940_2020.xlsx')
relevant=pd.read_excel('./031_recon/refs_1940_2020_top5.xlsx')

In [50]:
ids=list(set(cit_network['id_o'].astype(str)))
ids_m=list(set(cit_network['match_id'].astype(str)))



NameError: name 'j_data' is not defined

In [31]:
nodes=j_data[(j_data['id_o'].isin(ids)==True)|(j_data['id_o'].isin(ids_m)==True)][['id_o','journal', 
       'year', 'content_type']].drop_duplicates(keep='first').reset_index(drop=True)
nodes['id_o']=nodes['id_o'].astype(str)

In [46]:
nodes=nodes.rename(columns={'id_o':"Id"})
nodes.to_csv("./031_recon/nodes_alt_header.csv", index=None, encoding="UTF-8")


In [47]:
nodes.head()

,Id,journal,year,content_type
0,26966478,american economic review,2020,Article
1,26966479,american economic review,2020,Article
2,26966480,american economic review,2020,Article
3,26966481,american economic review,2020,Article
4,26966482,american economic review,2020,Article


In [34]:
cit_network["match_id"]=cit_network["match_id"].astype(str)
cit_network["id_o"]=cit_network["id_o"].astype(str)

In [35]:
edges=cit_network[(cit_network["id_o"]!=cit_network["match_id"]) & (cit_network["match_id"]!="CHECKREQ")].drop_duplicates(subset=['match_id',"id_o"], keep="first")

In [48]:
edges=edges.rename(columns={'id_o':"Source", 'match_id':'Target'})
edges.to_csv('./031_recon/edges_alt_header.csv', index=False)

In [39]:
nodes.shape

(24442, 4)

In [40]:
edges.shape

(107247, 3)

In [41]:
cit_network.shape

(108988, 3)

In [136]:
checks=list(cit_network[cit_network['match_id']=="CHECKREQ"]['id_o'])

In [75]:
inds=list(j_data[(j_data['year']>=1940)&(j_data['content_type']!='MISC')&(j_data['content_type']!='Review')]['id_o'])

In [137]:
n_ind=list(set(ids+ids_m+checks))

In [138]:
len(n_ind)

24443

In [139]:
len(inds)

30595

In [140]:
test=j_data[(j_data['id_o'].isin(n_ind)==False)&(j_data['content_type']!='MISC')&(j_data['content_type']!='Review')&(j_data['content_type']!='Review2')& (j_data['year']>=1940)]

In [141]:
test['content_type'].value_counts()

Article       5660
Discussion     511
Comment        367
Reply          342
Rejoinder       64
Name: content_type, dtype: int64

In [142]:
test['journal'].value_counts()

american economic review          4381
econometrica                       950
journal of political economy       890
quarterly journal of economics     447
review of economic studies         276
Name: journal, dtype: int64

In [143]:
# test[test['journal']=="american economic review"][['year', 'content_type']].value_counts()
# test[(test['journal']=="american economic review")&(test['year']==2010)]

list(test['id_o'].shape)

[6944]

In [144]:
j_inds=j_data[(j_data['content_type']!='MISC')&(j_data['content_type']!='Review')&(j_data['content_type']!='Review2')& (j_data['year']>=1940)]

In [145]:
cit_network.shape

(108988, 3)

In [146]:
edges.shape

(107247, 3)

In [147]:
cit_network.columns

Index(['id_o', 'match_id', 'f_key'], dtype='object')

In [148]:
relevant.shape

(108988, 31)

In [149]:
all_refs.shape

(534347, 31)

In [150]:
digi=list(set(all_refs['id_o']))

In [151]:
len(digi)

24043

In [157]:
test[test['id_o'].isin(digi)==False].head()

,issue_url,URL,journal,number,publisher,title,volume,year,abstract,author,pages,reviewed-author,uploaded,author_split,title_10,content_type,id_o,title_proc
241,https://www.jstor.org/stable/10.2307/e26636969,https://www.jstor.org/stable/26636977,american economic review,5,American Economic Association,Walk on the Wild Side: Temporarily Unstable Paths and Multiplicative Sunspots,109,2019,"We propose a generalization of the rational expectations framework to allow for temporarily unstable paths. Our approach introduces multiplicative sunspot shocks and it yields drifting parameters and stochastic volatility. Then, we provide an econometric strategy to estimate this generalized model on the data. The methodology allows the data to choose between different possible alternatives: determinacy, indeterminacy, and temporary instability. We apply our methodology to US inflation dynamics in the 1970s through the lens of a simple New Keynesian model. When temporarily unstable paths are allowed, the data unambiguously select them to explain the stagflation period in the 1970s.",Guido Ascari and Paolo Bonomolo and Hedibert F. Lopes,1805-1842,NaN,1.0,"['Guido Ascari', 'Paolo Bonomolo', 'Hedibert F. Lopes']",Walk on the Wild Side: Temporarily Unstable Paths and Multiplicative Sunspots,Article,26636977,walk on the wild side: temporarily unstable paths and multiplicative sunspots
253,https://www.jstor.org/stable/10.2307/e26637201,https://www.jstor.org/stable/26637207,american economic review,4,American Economic Association,Selling to Advised Buyers,109,2019,"In many cases, buyers are not informed about their valuations and rely on experts, who are informed but biased for overbidding. We study auction design when selling to such “advised buyers.” We show that a canonical dynamic auction, the English auction, has a natural equilibrium that outperforms standard static auctions in expected revenues and allocative efficiency. The ability to communicate as the auction proceeds allows for more informative communication and gives advisors the ability to persuade buyers into overbidding. The same outcome is the unique equilibrium of the English auction when bidders can commit to contracts with their advisors.",Andrey Malenko and Anton Tsoy,1323-1348,NaN,1.0,"['Andrey Malenko', 'Anton Tsoy']",Selling to Advised Buyers,Article,26637207,selling to advised buyers
377,https://www.jstor.org/stable/10.2307/e26528515,https://www.jstor.org/stable/26528521,american economic review,7,American Economic Association,The Twin Ds: Optimal Default and Devaluation,108,2018,"A salient characteristic of sovereign defaults is that they are typically accompanied by large devaluations. This paper presents new evidence of this empirical regularity known as the Twin Ds and proposes a model that rationalizes it as an optimal policy outcome. The model combines limited enforcement of debt contracts and downward nominal wage rigidity. Under optimal policy, default is shown to occur during contractions. The role of default is to free up resources for domestic absorption, and the role of exchange rate devaluation is to lower the real value of wages, thereby reducing involuntary unemployment.",Seunghoon Na and Stephanie Schmitt-Grohé and Martín Uribe and Vivian Yue,1773-1819,NaN,1.0,"['Seunghoon Na', 'Stephanie Schmitt-Grohé', 'Martín Uribe', 'Vivian Yue']",The Twin Ds: Optimal Default and Devaluation,Article,26528521,the twin ds: optimal default and devaluation
464,https://www.jstor.org/stable/10.2307/i40206841,https://www.jstor.org/stable/44871798,american economic review,11,American Economic Association,Gresham's Law of Model Averaging,107,2017,"A decision maker doubts the stationarity of his environment. In response, he uses two models, one with time-varying parameters, and another with constant parameters. Forecasts are then based on a Bayesian model averaging strategy, which mixes forecasts from the two models. In reality, structural parameters are constant, but the unknown) tru